<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/ELECTRA_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow==1.15
!git clone https://github.com/google-research/electra.git

     |████████████████████████████████| 412.3MB 18kB/s 
     |████████████████████████████████| 3.8MB 46.7MB/s 
     |████████████████████████████████| 512kB 44.9MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
Cloning into 'electra'...
remote: Enumerating objects: 104, done.
remote: Total 104 (delta 0), reused 0 (delta 0), pack-reused 104
Receiving objects: 100% (104/104), 96.57 KiB | 1.93 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [2]:
import os
import json
import pprint
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is => ', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is =>  grpc://10.62.93.226:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 15765868453811228445),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 6632495775336772580),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 4103762867967987213),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 6707915429204639678),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 14045506625766943625),
 _DeviceAttributes(/job:tpu_worker/replica:0/tas

In [3]:
BUCKET = 'electra_mals' #@param {type:"string"}
assert BUCKET, '*** Must specify an existing GCS bucket name ***'
data_dir_name = 'ELECTRA-TPU' #@param {type:"string"}
BUCKET_NAME = 'gs://{}'.format(BUCKET)
DATA_DIR = 'gs://{}/{}'.format(BUCKET,data_dir_name)
MODEL_NAME = 'ELECTRA_MALS'
print(os.environ['TPU_NAME'])

grpc://10.62.93.226:8470


In [9]:
hparams = {
    "do_train": "true",
    "do_eval": "false",
    "use_tpu": "true", 
    "num_tpu_cores": 8, 
    "tpu_name": "grpc://10.62.93.226:8470", 
    "model_size": "base",
    "do_lower_case": "false",
    "max_seq_length": 512,
    "vocab_size": 34100,
    "num_train_steps": 1_000_000,
    "save_checkpoints_steps": 1_000,
    "train_batch_size": 32,
}
           
with open("/content/drive/My Drive/Colab Notebooks/from_scratch/hparams.json", "w") as f:
    json.dump(hparams, f)

In [ ]:
!python3 electra/run_pretraining.py \
    --data-dir $BUCKET_NAME/ELECTRA-TPU/ \
    --model-name $MODEL_NAME \
    --hparams '/content/drive/My Drive/Colab Notebooks/from_scratch/hparams.json'

Config:
debug False
disallow_correct False
disc_weight 50.0
do_eval false
do_lower_case false
do_train true
electra_objective True
embedding_size None
eval_batch_size 128
gcp_project None
gen_weight 1.0
generator_hidden_size 0.25
generator_layers 1.0
iterations_per_loop 200
keep_checkpoint_max 5
learning_rate 0.0005
lr_decay_power 1.0
mask_prob 0.15
max_predictions_per_seq 79
max_seq_length 512
model_dir gs://electra_mals/ELECTRA-TPU/models/ELECTRA_MALS
model_hparam_overrides {}
model_name ELECTRA_MALS
model_size base
num_eval_steps 100
num_tpu_cores 8
num_train_steps 1000000
num_warmup_steps 10000
pretrain_tfrecords gs://electra_mals/ELECTRA-TPU/pretrain_tfrecords/pretrain_data.tfrecord*
results_pkl gs://electra_mals/ELECTRA-TPU/models/ELECTRA_MALS/results/unsup_results.pkl
results_txt gs://electra_mals/ELECTRA-TPU/models/ELECTRA_MALS/results/unsup_results.txt
save_checkpoints_steps 1000
temperature 1.0
tpu_job_name None
tpu_name grpc://10.62.93.226:8470
tpu_zone None
train_batch_size